In [2]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.special import beta
from scipy.special import comb
import qmcpy
import time
import pandas as pd
from scipy import stats
import math
import multiprocessing as mp

In [3]:
# sampling function
def int_sin_m(x, m):
    if m == 0:
        return x
    elif m == 1:
        return 1 - np.cos(x)
    else:
        return (m - 1) / m * int_sin_m(x, m - 2) - np.cos(x) * np.sin(x) ** (
                m - 1
        ) / m

def zero_sum_projection(d):
    basis = np.array([[1.0] * i + [-i] + [0.0] * (d - i - 1) for i in range(1, d)])
    return np.array([v / np.linalg.norm(v) for v in basis])


def bm_sphere_points(n, d):
    import qmcpy
    mod = d%2
    if mod == 1:
        dim = d+1
        pairs = dim//2
    else:
        dim = d
        pairs = dim//2
    X = qmcpy.Sobol(dim, graycode=True).gen_samples(n)
    Y = np.zeros((n, dim))
    for i in range(pairs):
        R = np.sqrt(-2 * np.log(X[:, 2*i]))
        Theta = 2 * np.pi *X[:, 2*i+1]
        Y[:, 2*i] = R * np.cos(Theta)
        Y[:, 2*i+1] = R * np.sin(Theta)
    Y = Y[:, :d]
    A = np.sum(Y**2, axis = 1)
    B = np.tile(A.reshape(n, 1), (1,d))
    Y = Y/np.sqrt(B)
    return Y

def tfww_sphere_points(n, d):
    import qmcpy
    X = qmcpy.Sobol(d-1, graycode=True).gen_samples(n)
    dim = d
    mod = dim%2
    if mod == 1:
        m = dim//2
        g = np.zeros((n, m+1))
        c = np.zeros((n, m))
        g[:, 0] = 0
        g[:, m] = 1
        for j in range(m-1):
            g[:, m-j-1] = g[:, m-j]*np.power(X[:, m-j-2],2/(2*(m-j-1)+1))
            c[:, m-j-1] = np.sqrt(g[:, m-j]-g[:, m-j-1])
        c[:, 0] = np.sqrt(g[:, 1] - g[:, 0])
        Y = np.zeros((n, dim))
        Y[:, 0] = c[:, 0]*(1-2*X[:, m-1])
        Y[:, 1] = 2*c[:, 0]*np.sqrt(X[:, m-1]*(1-X[:, m-1]))*np.cos(X[:, m] * 2 * np.pi)
        Y[:, 2] = 2*c[:, 0]*np.sqrt(X[:, m-1]*(1-X[:, m-1]))*np.sin(X[:, m] * 2 * np.pi)
        for l in range(m-1):
            Y[:, 2*l+3] = c[:, l+1]*np.cos(X[:, m+l+1] * 2 * np.pi)
            Y[:, 2*l+4] = c[:, l+1]*np.sin(X[:, m+l+1] * 2 * np.pi)
    else:
        m = dim//2
        g = np.zeros((n, m+1))
        c = np.zeros((n, m))
        g[:, 0] = 0
        g[:, m] = 1
        for j in range(m-1):
            g[:, m-j-1] = g[:, m-j]*np.power(X[:, m-j-2],1/(m-j-1))
            c[:, m-j-1] = np.sqrt(g[:, m-j]-g[:, m-j-1])
        c[:, 0] = np.sqrt(g[:, 1] - g[:, 0])
        Y = np.zeros((n, d))
        for l in range(m):
            Y[:, 2*l] = c[:, l]*np.cos(X[:, m+l-1] * 2 * np.pi)
            Y[:, 2*l+1] = c[:, l]*np.sin(X[:, m+l-1] * 2 * np.pi)
    return Y

def sct_sphere_points(n, d):
    import qmcpy
    X = qmcpy.Sobol(d - 1, graycode=True).gen_samples(n)
    n = X.shape[0]
    Y = np.ones((n, d))
    for i in range(n):
        Y[i][0] *= np.sin(X[i, 0] * 2 * np.pi)
        Y[i][1] *= np.cos(X[i, 0] * 2 * np.pi)

    for j in range(2, d):
        inv_beta = 1 / beta(j / 2, 1 / 2)
        for i in range(n):
            root_function = lambda varphi: inv_beta * int_sin_m(varphi, j - 1) - X[i, j - 1]
            deg = root_scalar(root_function, bracket=[0, np.pi], xtol=1e-15).root
            for k in range(j):
                Y[i][k] *= np.sin(deg)
            Y[i][j] *= np.cos(deg)
    return Y

def gen_sobol_permutations(n, d, method):
    if method == "bm":
        sphere_points = bm_sphere_points(n, d - 1)
    elif method == "tfww":
        sphere_points = tfww_sphere_points(n, d - 1)
    else:
        sphere_points = sct_sphere_points(n, d - 1)
    basis = zero_sum_projection(d)
    projected_sphere_points = sphere_points.dot(basis)
    p = np.zeros((n, d), dtype=np.int64)
    for i in range(n):
        p[i] = np.argsort(projected_sphere_points[i])
    return p


In [4]:
# generate the model parameters
class bayesian_network_posterior:
    def __init__(self, the_R, tau2_R, kap_R, lam_R, mu, num_state, num_action, n_time, beta, v2):
        self.the_R = the_R
        self.tau2_R = tau2_R
        self.kap_R = kap_R
        self.lam_R = lam_R
        self.num_nodes = (num_state + num_action) * n_time
        self.num_state = num_state
        self.num_action = num_action
        self.mu = mu
        self.beta = beta
        self.v2 = v2
        self.n_time = n_time

    def posterior_sample(self, size=1, useFixed = True):
        p_beta = np.zeros(shape=(self.num_nodes,self.num_nodes, size))
        p_v2 = np.zeros(shape=(self.num_nodes, size))
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if self.tau2_R[i, j] != 0:
                    p_beta[i, j, ] = np.random.normal(loc=self.the_R[i,j], scale=np.sqrt(self.tau2_R[i,j]), size=size)
            gamma_rate = self.lam_R[i] / 2
            p_v2[i,] = 1 / np.random.gamma(shape=self.kap_R[i] / 2, scale=1/gamma_rate, size=size)
        if useFixed:
            p_beta = self.beta
            p_v2 = self.v2
        return (p_beta, p_v2, self.mu)

class bayesian_network:
    def __init__(self, mu, beta, v2, num_action, num_state, n_time, normalized = True, sample_mean=None, sample_sd=None):
        self.n_time = n_time
        if normalized:
            self.sample_mean = sample_mean
            self.sample_sd = sample_sd
            self.normalized = True
        self.initial_state_full = np.array([0.05, 0.00, 0.00, 30.00, 5.00,0.7])
        if num_state == 4:
            self.initial_state_base = self.initial_state_full[[0, 1, 3, 4, 5]]
        if num_state == 5:
            self.initial_state_base = self.initial_state_full
        self.mu = mu
        self.v2 = v2
        self.beta = beta
        self.num_action = num_action
        self.num_state = num_state
        self.n_factor = num_action + num_state
        self.beta_state = np.zeros(shape=(n_time, num_state, num_state)) # s -> s
        self.beta_action = np.zeros(shape=(n_time, num_action, num_state)) # a -> s
        for i in range(n_time-1):
            self.beta_state[i,:,:] = beta[(self.n_factor * (i+1) - num_state):(self.n_factor * (i+1)), (self.n_factor * (i + 2) - num_state): (self.n_factor * (i + 2))]
            self.beta_action[i,:,:] = beta[(self.n_factor * i):(self.n_factor * i + self.num_action), (self.n_factor * (i + 2) - num_state):(self.n_factor * (i + 2))]
        self.mu_a = []
        for i in range(self.n_time):
            temp_list = []
            for j in range(self.num_action):
                temp_list.append(self.mu[i * self.n_factor + j])
            self.mu_a.append(temp_list)

    def initial_state_generator(self, scale=10):
        init_states = self.initial_state_base + np.abs(np.random.normal(0, np.array(self.initial_state_base)/scale + 0.01))
        init_states = init_states[[0,1,3,4,5]] # init_states[:-1] * init_states[-1]
        self.initial_state = (init_states - self.sample_mean[(self.n_factor - self.num_state):self.n_factor]) / self.sample_sd[(self.n_factor - self.num_state):self.n_factor]

    def rescale_action(self, action, t, scale_method = "standard"):
        if not self.normalized:
            return action
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action + self.sample_mean[(self.n_factor * t):(self.n_factor * t + self.num_action)]
        else:
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action

    def rescale_state(self, state, t, scale_method = "standard"):
        if not self.normalized:
            return state
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state + self.sample_mean[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))]
        else:
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state
        
beta_gibbs = pd.read_csv('data/beta_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
v2_gibbs = pd.read_csv('data/v2_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None,
                        dtype=np.float64)
the_R = pd.read_csv('data/the_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
tau2_R = pd.read_csv('data/tau2_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
kap_R = pd.read_csv('data/kap_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
lam_R = pd.read_csv('data/lam_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
sd = pd.read_csv('data/sd_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
bn_post = bayesian_network_posterior(the_R.to_numpy(), tau2_R.to_numpy(), kap_R.to_numpy(), lam_R.to_numpy(),
                                         mu.to_numpy().flatten(), 5, 1, 36, beta_gibbs, v2_gibbs)
# mua = np.array(mu.to_numpy().reshape(36, 6)[:, 0]).reshape(1, 36)
# mus = np.array(mu.to_numpy().reshape(36, 6)[:, 1:]).reshape(5, 36)

def simulate(H = 36, nums = 5, seed = 1):
    np.random.seed(seed)
    V_mat = np.zeros((H*nums, H*nums))
    mat = (2*np.random.rand((H-1)*nums, (H-1)*nums)-1)/5
    V_mat[nums*1:nums*H, nums*1:nums*H] = mat.dot(mat.T)
    return V_mat

In [5]:
def rescale_action(action, t, mu, sd, scale_method = "standard"):
    mu0 = mu.to_numpy().flatten()
    sample_sd = sd.to_numpy().flatten()
    a_t = sample_sd[(6 * t):(6 * t + 1)] * action + mu0[(6 * t):(6 * t + 1)]
    return a_t

# mean function and variance function
def cal_mean(s1, mus, mua, mu, sd, betaa, betas, theta, b_r, c_r):
    H = np.shape(mus)[1]
    nums =np.shape(mus)[0]
    s = np.zeros((nums, H-1))
    a = np.zeros(H)
    R = np.identity(nums)
    theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)
    m_r = 0
    for i in range(H-1):
        coe = np.transpose(betas[:,:,i] + theta[:,i].reshape(nums, 1).dot(betaa[:,i].reshape(1, nums)))
        R = coe.dot(R)
        si = mus[:, i+1].reshape(nums, 1) + R.dot(s1.reshape(nums,1) - mus[:, 0].reshape(nums,1))
        s[:, i] = si.reshape(nums)
    s = np.concatenate((s1.reshape(nums, 1), s), axis = 1)
    for i in range(H-1):
        temp_a = mua[0, i] + np.sum(theta_0[:, i]*(s[:, i]-mus[:, i]))
        a[i] = rescale_action(temp_a, i, mu, sd)
        m_r = m_r-534.52*a[i]
    m_r = m_r + np.sum(1.29*s[1,H-1])
    return [m_r, s]

def cal_var(betaa, betas, theta, b_r, c_r, V_mat):
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1]+1    
    # calculate the R matrix
    R = np.zeros(((H-1)*nums, (H-1)*nums))
    for t2 in range(H-1):
        R0 = np.identity(nums)
        for t1 in range(0, t2+1):
            R0 = R0.dot(np.transpose(betas[:,:, t2-t1]+theta[:, t2-t1].reshape(nums, 1).dot(betaa[:, t2-t1].reshape(1,nums))))
            R[nums*(t2-t1):nums*(t2-t1+1), nums*t2:nums*(t2+1)] = R0
    # calculate the covariance matrix
    A1 = np.zeros((H*nums, H*nums))
    A2 = np.zeros((H*nums, H*nums))
    A3 = np.zeros((H*nums, H*nums))
    cov = np.zeros((H*nums, H*nums))
    for t2 in range(H-1):
        for t1 in range(t2+1): 
            A1[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)] = (A1[nums*(t1+1):nums*(t1+2), nums*t2:nums*(t2+1)]+A2[nums*(t1+1):nums*(t1+2), nums*t2:nums*(t2+1)]).dot(R[nums*t2:nums*(t2+1), nums*t2:nums*(t2+1)].T)
            A1[nums*(t2+1):nums*(t2+2), nums*(t1+1):nums*(t1+2)] = A1[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)].T
            A2[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)] = R[nums*t1:nums*(t1+1), nums*t1:nums*(t1+1)].dot(A2[nums*(t1):nums*(t1+1), nums*(t2+1):nums*(t2+2)]+V_mat[nums*(t1):nums*(t1+1), nums*(t2+1):nums*(t2+2)])
            A2[nums*(t2+1):nums*(t2+2), nums*(t1+1):nums*(t1+2)] = A2[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)].T
            A3[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)] = (A3[nums*(t1+1):nums*(t1+2), nums*(t2):nums*(t2+1)]+V_mat[nums*(t1+1):nums*(t1+2), nums*(t2):nums*(t2+1)]).dot(R[nums*t2:nums*(t2+1), nums*t2:nums*(t2+1)].T)
            A3[nums*(t2+1):nums*(t2+2), nums*(t1+1):nums*(t1+2)] = A3[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)].T
            cov[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)] = A1[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]+A2[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]+A3[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]+V_mat[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]
            cov[nums*(t2+1):nums*(t2+2), nums*(t1+1):nums*(t1+2)] = cov[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]
    # calculate the variance of the cumulative rewards
    v_r = 0
    theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)
    for t2 in range(H-1):
        for t1 in range(H-1):
            alpha1 = (b_r[t1+1]*(theta_0[:,t1+1].reshape(1, nums))+c_r[:, t1+1].reshape(1,nums))/400
            alpha2 = (b_r[t2+1]*(theta_0[:,t2+1].reshape(1, nums))+c_r[:, t2+1].reshape(1,nums))/400        
            v_r = v_r+alpha1.dot(cov[nums*(t1+1):nums*(t1+2), nums*(t2+1):nums*(t2+2)]).dot(alpha2.T)
    return [v_r, cov]

# QMC for shapley value
def QMC_mean(s1, mus, mua, mu, sd, betaa, betas, theta, b_r, c_r, P):
    # calculate approximate shapley value using QMC
    m = np.shape(P)[0]
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1]+1
    num_th = nums*(H-1)
    shap_value_r = np.zeros((num_th))
    shap_value_s = np.zeros((num_th, nums, H))
    theta_flat = theta.flatten()
    for i in range(m):
        permute = P[i,:]
        temp = np.zeros(num_th)
        [r_prev, s_prev] = cal_mean(s1, mus, mua, mu, sd, betaa, betas, temp.reshape(nums, H-1), b_r, c_r)
        for j in range(num_th):
            index = permute[j]
            temp[index] = theta_flat[index].copy()
            [r, s] = cal_mean(s1, mus, mua, mu, sd, betaa, betas, temp.reshape(nums, H-1), b_r, c_r)
            shap_value_r[index] = shap_value_r[index]+(r - r_prev)
            shap_value_s[index,:,:] = shap_value_s[index]+(s - s_prev)
            [r_prev, s_prev] = [r.copy(), s.copy()]
#             shap_value[index, :, :] = shap_value[index, :, :] + (v1 - v0).reshape(nums, H-1)
    shap_value_r = shap_value_r/m
    shap_value_s = shap_value_s/m    
    return [shap_value_r, shap_value_s]

def QMC_var(betaa, betas, theta, b_r, c_r, V_mat, P):
    # calculate approximate shapley value using QMC
    m = np.shape(P)[0]
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1]+1
    num_th = nums*(H-1)
    shap_value_r = np.zeros((num_th))
    shap_value_s = np.zeros((num_th, nums, H))
    theta_flat = theta.flatten()
    for i in range(m):
        permute = P[i,:]
        temp = np.zeros(num_th)
        for j in range(num_th):
            index = permute[j]
            [vr_prev, cov_prev] = cal_var(betaa, betas, temp.reshape(nums, H-1),b_r, c_r, V_mat)
            vs_prev = cov_prev.diagonal().reshape(nums,H,order='F')
            temp[index] = theta_flat[index].copy()
            [vr, cov] = cal_var(betaa, betas, temp.reshape(nums, H-1),b_r, c_r, V_mat)
            vs = cov.diagonal().reshape(nums,H,order='F')
            shap_value_r[index] = shap_value_r[index]+(vr - vr_prev)
            shap_value_s[index,:,:] = shap_value_s[index]+(vs - vs_prev)
#             shap_value[index, :, :] = shap_value[index, :, :] + (v1 - v0).reshape(nums, H-1)
    shap_value_r = shap_value_r/m
    shap_value_s = shap_value_s/m    
    return [shap_value_r, shap_value_s]

def cal_mean_theta(p_beta0, p_v20, mu0, mu, sd, s1, theta, b_r, c_r, P):
    beta = p_beta0
    v2 = p_v20
    bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
    mua = np.array(bn.mu_a).reshape(1, 36)
    mus = np.array(bn.mu.reshape(36, 6)[:,1:]).reshape(5,36)
    betas = np.zeros((5, 5, 36))
    for i in range(36):
        betas[:,:,i] = bn.beta_state[i,:,:]
    betaa = np.transpose(bn.beta_action.reshape(36,5))
    [shap_mean_r, shap_mean_s] = QMC_mean(s1, mus, mua, mu, sd, betaa, betas, theta, b_r, c_r, P)
    return [shap_mean_r, shap_mean_s]

def cal_var_theta(p_beta0, p_v20, mu0, mu, sd, theta, b_r, c_r, V_mat, P):
    beta = p_beta0
    v2 = p_v20
    k = 0
    for i in range(216):
        if i%6 != 0:
            V_mat[k,k] = v2[i]
            k = k+1   
    bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
    betas = np.zeros((5, 5, 36))
    for i in range(36):
        betas[:,:,i] = bn.beta_state[i,:,:]
    betaa = np.transpose(bn.beta_action.reshape(36,5))
    [shap_var_r, shap_var_s] = QMC_var(betaa, betas, theta, b_r, c_r, V_mat, P)
    return [shap_var_r, shap_var_s]


In [ ]:
# calculate the predictive shapley value
# sample from the parameters
ssize1 = 200
p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize1, useFixed=False)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
s1 = np.array([0.05,0,30,5,0.7])
theta = np.load('theta.npy')
b_r = np.repeat(-534.52, 36)
c_r = np.zeros((5, 36))
c_r[1, 35] = 1.29
m1 = 500
nums = 5
H = 36

# sample the permutations
P = np.zeros((ssize1, m1, nums * (H-1)), dtype=np.int64)
for i in range(ssize1):
    P_i = gen_sobol_permutations(m1, nums * (H-1), "tfww")
    P[i, :, :] = P_i

start = time.time()
print(1)
shap_mean_theta_list = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize1):
    shap_mean_theta_list.append(pool.apply_async(cal_mean_theta, args = (p_beta[:, :, i], p_v2[:, i], mu0, mu, sd, s1, theta, b_r, c_r, P[i, :, :])))
pool.close()
pool.join()
end = time.time()
print(end - start)


shap_mean_r_samples = np.zeros((ssize1, 175))
shap_mean_s_samples = np.zeros((ssize1, 175, nums, H))
for i in range(ssize1):
    shap_mean_r_samples[i, :] = shap_mean_theta_list[i].get()[0]
    shap_mean_s_samples[i, :, :, :] = shap_mean_theta_list[i].get()[1]

np.save('linear_shap_mean_r_theta_samples.npy', shap_mean_r_samples)
np.save('linear_shap_mean_s_theta_samples.npy', shap_mean_s_samples)

In [ ]:
# calculate the variance-based Shapley value

V_mat = simulate(H = 36, nums = 5, seed = 1)

# calculate the variance-based shapley value
# sample from the parameters
ssize2 = 200
m2 = 500
p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize2, useFixed=False)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
s1 = np.array([0.00000000e+00,0.00000000e+00,3.00000000e+01,5.00000000e+00,7.00000000e-01])
theta = np.load('theta.npy')
b_r = np.repeat(-534.52, 36)
c_r = np.zeros((5, 36))
c_r[1, 35] = 1.29


# sample the permutations
P = np.zeros((ssize2, m2, nums * (H-1)), dtype=np.int64)
for i in range(ssize2):
    P_i = gen_sobol_permutations(m2, nums * (H-1), "tfww")
    P[i, :, :] = P_i

start = time.time()
print(1)
shap_var_theta_list = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize2):
    shap_var_theta_list.append(pool.apply_async(cal_var_theta, args = (p_beta[:, :, i], p_v2[:, i], mu0, mu, sd, theta, b_r, c_r, V_mat, P[i, :, :])))
pool.close()
pool.join()
end = time.time()
print(end - start)

shap_var_r_samples = np.zeros((ssize2, 175))
shap_var_s_samples = np.zeros((ssize2, 175, nums, H))
for i in range(ssize2):
    shap_var_r_samples[i, :] = shap_var_theta_list[i].get()[0]
    shap_var_s_samples[i, :, :, :] = shap_var_theta_list[i].get()[1]

np.save('linear_shap_var_r_theta_samples.npy', shap_var_r_samples)
np.save('linear_shap_var_s_theta_samples.npy', shap_var_s_samples)

In [9]:
def cal_mean_random(s1, mu0, mu, sd, p_beta0, v20, theta, b_r, c_r):
    v2 = np.array(v20.reshape(36, 6)[:, 1:]).reshape(5, 36)
    beta = p_beta0
    bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
    betas = np.zeros((5, 5, 36))
    for i in range(36):
        betas[:,:,i] = bn.beta_state[i,:,:]
    betaa = np.transpose(bn.beta_action.reshape(36,5))
    e = np.sqrt(v2)/200
    H = 36
    nums = 5
    s = np.zeros((nums, H-1))
    a = np.zeros(H)
    R = np.zeros((H, H, nums, nums))
    R0 = np.zeros((H-1, nums, nums))
    theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)
    for i in range(H-1):
        coe = np.transpose(betas[:,:,i] + theta_0[:,i].reshape(nums, 1).dot(betaa[:,i].reshape(1, nums)))
        R0[i, :, :] = coe.copy()
    for i in range(H-1):
        R_temp = np.identity(nums)
        R[i, i, :, :] = R_temp
        for j in range(i+1, H):
            R_temp = R0[j-1, :, :].dot(R_temp)
            R[i, j, :, :] = R_temp.copy()
    R[H-1, H-1, :, :] = np.identity(nums)
    shap_s = np.zeros((nums, H, nums, H))
    for i in range(H):
        for j in range(nums):
            indicator = np.zeros(nums)
            indicator[j] = 1
            for k in range(i, H):
                shap_s[j, i, :, k] = R[i, k, :, :].dot(e[j, i]*indicator.reshape(nums, 1)).reshape(nums)
    shap_r = np.zeros((nums, H))
    alpha = np.zeros((nums, H))
    for i in range(H):
        alpha[:, i] = (b_r[i] * theta_0[:,i].reshape(nums, 1) + c_r[:, i].reshape(nums, 1)).reshape(nums)
    for i in range(nums):
        indicator = np.zeros(nums)
        indicator[i] = 1
        for j in range(H):
            for t in range(j, H):
                shap_r[i, j] = shap_r[i, j] + alpha[:, t].reshape(1, nums).dot(R[j, t, :, :]).dot(indicator.reshape(nums, 1).dot(e[i, j].reshape(1,1)))
    return [shap_r, shap_s]

def cal_var_random(s1, p_beta0, v20, theta, b_r, c_r, V_mat):
    v2 = v20
    beta = p_beta0
    bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
    betas = np.zeros((5, 5, 36))
    for i in range(36):
        betas[:,:,i] = bn.beta_state[i,:,:]
    betaa = np.transpose(bn.beta_action.reshape(36,5))
    k = 0
    for i in range(216):
        if i%6 != 0:
            V_mat[k,k] = v2[i]
            k = k+1   
    H = 36
    nums = 5
    theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)
    R1 = np.zeros((H, H, nums, nums))
    R0 = np.zeros((H-1, nums, nums))
    for i in range(H-1):
        coe = np.transpose(betas[:,:,i] + theta_0[:,i].reshape(nums, 1).dot(betaa[:,i].reshape(1, nums)))
        R0[i, :, :] = coe.copy()
    for i in range(H-1):
        R_temp = np.identity(nums)
        R1[i, i, :, :] = R_temp
        for j in range(i+1, H):
            R_temp = R0[j-1, :, :].dot(R_temp)
            R1[i, j, :, :] = R_temp.copy()
    R1[H-1, H-1, :, :] = np.identity(nums)
    shap_s = np.zeros((nums, H, nums, H))
    for i in range(nums):
        for j in range(H):
            for k in range(j, H):
                Rt = np.zeros((nums, (k+1)*nums))
                for t in range(k):
                    Rt[:, t*nums : (t+1)*nums] = R1[t, k, :, :]
                Vt = V_mat[0 : nums*(k+1), 0 : nums*(k+1)]
                l = j * nums + i
                indicator = np.zeros((nums*(k+1), nums*(k+1)))
                indicator[:, l] = 1
                shap_s[i, j, :, k] = np.diag(Rt.dot(0.5*Vt*(indicator+indicator.T)).dot(Rt.T))
    shap_r = np.zeros((nums, H))
    alpha = np.zeros((nums, H))
    for i in range(H):
        alpha[:, i] = (b_r[i] * theta_0[:,i].reshape(nums, 1) + c_r[:, i].reshape(nums, 1)).reshape(nums)/400
    for i in range(nums):
        for j in range(H):
            shap_r1 = 0
            for k in range(j, H):
                Rt = np.zeros((nums, (k+1)*nums))
                for t in range(k):
                    Rt[:, t*nums : (t+1)*nums] = R1[t, k, :, :]
                Vt = V_mat[0 : nums*(k+1), 0 : nums*(k+1)]
                indicator = np.zeros((nums*(k+1), nums*(k+1)))
                l = j * nums + i
                indicator[:, l] = 1
                shap_r1 = shap_r1 + alpha[:, k].reshape(1, nums).dot(Rt).dot(0.5*Vt*(indicator+indicator.T)).dot(Rt.T).dot(alpha[:, k].reshape(nums, 1))
            shap_r2 = 0
            for t2 in range(j, H):
                Rt2 = np.zeros((nums, (t2+1)*nums))
                for t in range(t2):
                    Rt2[:, t*nums : (t+1)*nums] = R1[t, t2, :, :]
                for t1 in range(j-1):
                    Rt1 = np.zeros((nums, (t1+1)*nums))
                    for t in range(t1):
                        Rt2[:, t*nums : (t+1)*nums] = R1[t, t1, :, :]
                    indicator = np.zeros((nums*(t1+1), nums*(t2+1)))
                    l = j * nums + i
                    indicator[:, l] = 1
                    Vt = V_mat[0 : nums*(t1+1), 0 : nums*(t2+1)]
                    shap_r2 = shap_r2 + alpha[:, t1].reshape(1, nums).dot(Rt1).dot(0.5*Vt*(indicator)).dot(Rt2.T).dot(alpha[:, t2].reshape(nums, 1))
            shap_r3 = 0
            for t2 in range(j, H):
                Rt2 = np.zeros((nums, (t2+1)*nums))
                for t in range(t2):
                    Rt2[:, t*nums : (t+1)*nums] = R1[t, t2, :, :]
                for t1 in range(j, t2):
                    Rt1 = np.zeros((nums, (t1+1)*nums))
                    for t in range(t1):
                        Rt2[:, t*nums : (t+1)*nums] = R1[t, t1, :, :]
                    indicator1 = np.zeros((nums*(t1+1), nums*(t2+1)))
                    l = j * nums + i
                    indicator1[:, l] = 1
                    indicator2 = np.zeros((nums*(t2+1), nums*(t1+1)))
                    indicator2[:, l] = 1
                    Vt = V_mat[0 : nums*(t1+1), 0 : nums*(t2+1)]
                    shap_r3 = shap_r3 + alpha[:, t1].reshape(1, nums).dot(Rt1).dot(0.5*Vt*(indicator1 + indicator2.T)).dot(Rt2.T).dot(alpha[:, t2].reshape(nums, 1))
            shap_r[i, j] = shap_r1 + shap_r2 + shap_r3
    return [shap_r, shap_s]


In [10]:
# predictive analysis for random inputs
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
ssize3 = 1000
p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize3, useFixed=False)
s1 = np.array([0.00000000e+00,0.00000000e+00,3.00000000e+01,5.00000000e+00,7.00000000e-01])
theta = np.load('theta.npy')
b_r = np.repeat(-534.52, 36)
c_r = np.zeros((5, 36))
c_r[1, 35] = 1.29

start = time.time()
print(1)
shap_mean_e_list = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize3):
    shap_mean_e_list.append(pool.apply_async(cal_mean_random, args = (s1, mu0, mu, sd, p_beta[:, :, i], p_v2[:, i], theta, b_r, c_r)))
pool.close()
pool.join()
end = time.time()
print(end - start)

shap_s_mean_e_samples = np.zeros((ssize3, nums, H, nums, H))
shap_r_mean_e_samples = np.zeros((ssize3, nums, H))
for i in range(ssize3):
    shap_r_mean_e_samples[i, :, :] = shap_mean_e_list[i].get()[0]
    shap_s_mean_e_samples[i, :, :, :, :] = shap_mean_e_list[i].get()[1]

np.save('linear_shap_r_mean_e_samples.npy', shap_r_mean_e_samples)
np.save('linear_shap_s_mean_e_samples.npy', shap_s_mean_e_samples)

1
2.7572696208953857


In [11]:
# variance based analysis for random inputs
V_mat = simulate(H = 36, nums = 5, seed = 1)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
ssize4 = 1000
p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize4, useFixed=False)
s1 = np.array([0.00000000e+00,0.00000000e+00,3.00000000e+01,5.00000000e+00,7.00000000e-01])
theta = np.load('theta.npy')
b_r = np.repeat(-534.52, 36)
c_r = np.zeros((5, 36))
c_r[1, 35] = 1.29

start = time.time()
print(1)
shap_var_e_list = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize4):
    shap_var_e_list.append(pool.apply_async(cal_var_random, args = (s1, p_beta[:,:,i], p_v2[:,i], theta, b_r, c_r, V_mat)))
pool.close()
pool.join()
end = time.time()
print(end - start)

shap_s_var_e_samples = np.zeros((ssize4, nums, H, nums, H))
shap_r_var_e_samples = np.zeros((ssize4, nums, H))
for i in range(ssize4):
    shap_r_var_e_samples[i, :, :] = shap_var_e_list[i].get()[0]
    shap_s_var_e_samples[i, :, :, :, :] = shap_var_e_list[i].get()[1]

np.save('linear_shap_r_var_e_samples.npy', shap_r_var_e_samples)
np.save('linear_shap_s_var_e_samples.npy', shap_s_var_e_samples)

1
2643.5484766960144
